# TM10007 Assignment ADNI Dataset
Bram Gerritse 4462599  
Saskia Bijl 4439457  
Karan Ramsodit  
Enzo Kerkhof 44488555

In [1]:
# Run this to use from colab environment to install git repo
!pip install -q --upgrade git+https://github.com/Enzo-Kerkhof/TM10007_PROJECT.git@random_forest

# Use the @ to determine what branch to install. Don't use to install master.

In [2]:
# Import used libraries
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import decomposition
from sklearn import neighbors
from sklearn import svm
from sklearn import metrics
from sklearn import feature_selection
from sklearn.linear_model import Lasso

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [3]:
# Data loading functions. Uncomment the one you want to use
from adni.load_data import load_data

data = load_data()
print('The dataset ABNI contains the following\n')
print(f'The number of samples/patients: {len(data.index)}')
print(f'The number of columns/features: {len(data.columns)}')

The dataset ABNI contains the following

The number of samples/patients: 855
The number of columns/features: 268


## Data preprocessing

In [0]:
# Drop labels from dataframe
X = data.drop(['label'],axis=1)

# Binerize labels AD = 1, CN = 0
Y = preprocessing.label_binarize(data['label'], ['CN', 'AD'])
Y = [i[0] for i in Y]

# Split data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.8)
y_train = np.array(y_train)

In [0]:
# Scale the data to be normal
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [0]:
# Perform a PCA
pca = decomposition.PCA(n_components=100)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [7]:
# Now first use the selectfrom model module. Select all features with a weight above the median.
selector = feature_selection.SelectFromModel(estimator=Lasso(alpha=10**(-10)), threshold='median')
selector.fit(X_train_scaled, y_train)
n_original = X_train_scaled.shape[1]
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)
n_selected = X_train_selected.shape[1]
print(f"Selected {n_selected} from {n_original} features.")


X_train = X_train_selected
X_test = X_test_selected

Selected 134 from 267 features.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0053341334447171195, tolerance: 0.003923976608187134
  positive)


## Hyperparameters

In [0]:
# Let us first see how much of a difference the number of trees, n_estimators, makes
n_trees = [50]
    
# Now use the classifiers on all datasets
for n_tree in n_trees:
  for X, Y in zip(X_train, y_train):
    clf = RandomForestClassifier(n_estimators=n_tree)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)

In [0]:
# Next, let us try to see how bootstrapping influences the process.
# Let us first see how much of a difference the number of trees, n_estimators, makes
clsfs = [RandomForestClassifier(n_estimators=50, bootstrap=False)]
    
# Now use the classifiers on all datasets
for clf in clsfs:
    for X, Y in zip(X_train, y_train):
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_train)

In [0]:
# Lastly, if you have an imbalance in your dataset, or one class is more important than the other, you may want
# to alter the class weigh in the random forest.
clsfs = [RandomForestClassifier(class_weight={0: 1, 1: 0.001}),
         RandomForestClassifier(class_weight={0: 1, 1: 1}),
         RandomForestClassifier(class_weight={0: 1, 1: 10}),
         RandomForestClassifier(class_weight={0: 1, 1: 100})]
    
# Now use the classifiers on all datasets
for clf in clsfs:
    for X, Y in zip(X_train, y_train):
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_train)